In [8]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Load Data

In [9]:
# Load the data
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')


train.Id = train.Id.astype(str)
test.Id = test.Id.astype(str)

for i in train.Id:
    if len(str(i)) < 4:
        train.Id = train.Id.replace(i, '0'*(4-len(str(i))) + str(i)).astype(str)

for i in test.Id:
    if len(str(i)) < 4:
        test.Id = test.Id.replace(i, '0'*(4-len(str(i))) + str(i)).astype(str)

def append_ext(fn):
    return fn+".jpg"

train["Id"]=train["Id"].apply(append_ext)
test["Id"]=test["Id"].apply(append_ext)

classes = train['label'].unique().tolist()



In [ ]:
train.head()

In [ ]:
print(f'Total number of classes: {train["label"].nunique()}')
print(f'Total number of images for training: {train.shape[0]}')
print(f'Total number of images for testing: {test.shape[0]}')
print(f'Total train + test images: {train.shape[0] + test.shape[0]}')

import os
print(f'Total number of images in the images folder: {len(os.listdir("images"))}')

In [ ]:
print('Number of images per class:')
train['label'].value_counts().plot(kind='bar')

In [ ]:
# Display single image for each class
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#import seaborn as sns
%matplotlib inline

plt.figure(figsize=(20, 20))

for i, c in enumerate(classes):
    plt.subplot(5, 2, i+1)
    img = mpimg.imread('images/' + train[train['label'] == c]['Id'].values[0])
    plt.imshow(img)
    plt.title(f'Label: {c}')
    plt.axis('off')
    i += 1
plt.show()

# Predict Pretrained Model

In [ ]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.xception import decode_predictions
from tensorflow.keras.preprocessing.image import load_img
import numpy as np

model = Xception(weights='imagenet', input_shape=(299, 299, 3))

x = load_img('images/0001.jpg', target_size=(299, 299))
x = np.array(x)
X = np.expand_dims(x, axis=0)
X = preprocess_input(X)

# display the image
plt.imshow(x/255.)
plt.show()

pred = model.predict(X)
decode_predictions(pred, top=3)[0]

# Training a model to classify kitchen image

In [ ]:
# Training baseline model with Xception
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.xception import decode_predictions
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2
)

train_generator = train_gen.flow_from_dataframe(
    dataframe=train,
    directory='images/',
    x_col='Id',
    y_col='label',
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = train_gen.flow_from_dataframe(
    dataframe=train,
    directory='images/',
    x_col='Id',
    y_col='label',
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# save train_generator.class_indices to a file
import json
with open('./models/class_indices.json', 'w') as f:
    json.dump(train_generator.class_indices, f)


In [16]:
base_model = Xception(weights='imagenet', include_top=False)

base_model.trainable = False

inputs = keras.Input(shape=(299, 299, 3))

base = base_model(inputs, training=False)

vectors = GlobalAveragePooling2D()(base)

outputs = Dense(6, activation='softmax')(vectors)

model = Model(inputs, outputs)


In [ ]:
learning_rate = 0.0001
epochs = 10
batch_size = 32
optimizer = Adam(lr=learning_rate)
loss = keras.losses.CategoricalCrossentropy()
metrics = ['accuracy']

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

model.summary()

In [ ]:
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator,
    callbacks=[
        ModelCheckpoint('./models/model.h', save_best_only=True),
        EarlyStopping(patience=3)
    ]
)



In [ ]:
# plot the training loss and accuracy
import matplotlib.pyplot as plt
plt.style.use("ggplot")
plt.figure()
N = epochs
plt.plot(np.arange(0, N), history.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), history.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), history.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")

In [ ]:
# predict on test set
test_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

test_generator = test_gen.flow_from_dataframe(
    dataframe=test,
    directory='images/',
    x_col='Id',
    y_col=None,
    target_size=(299, 299),
    batch_size=32,
    class_mode=None,
    shuffle=False
)

model = keras.models.load_model('model.h)

pred = model.predict(test_generator)

# decode predictions
pred = np.argmax(pred, axis=1)

# class labels
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in pred]

# create submission file
sub = pd.DataFrame({'Id': test.Id, 'label': predictions})
sub.to_csv('submission.csv', index=False)


In [16]:
# check for all test images in images folder
test = pd.read_csv('./data/test.csv')

test.Id = test.Id.astype(str)

for i in test.Id:
    if len(str(i)) < 4:
        test.Id = test.Id.replace(i, '0'*(4-len(str(i))) + str(i)).astype(str)

def append_ext(fn):
    return fn+".jpg"

test["Id"]=test["Id"].apply(append_ext)


import os
count = 0
for i in test.Id:
    if i not in os.listdir('images/'):
        count += 1
        print(i)

# Predict single image

In [ ]:
def predict_single(image_url, web=True):
    from tensorflow import keras
    import numpy as np
    import json

    # predict from image url from the web
    import requests
    from PIL import Image
    from io import BytesIO

    if web:
        response = requests.get(image_url)
        img = Image.open(BytesIO(response.content))
    else:
        img = Image.open(image_url)

    # preprocess image
    from tensorflow.keras.applications.xception import preprocess_input
    img = img.resize((299, 299))
    img = np.array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)

    # load model
    model = keras.models.load_model('./models/model_kaggle.h')

    # load class indices
    with open('./models/class_indices.json', 'r') as f:
        class_indices = json.load(f)

    # predict and decode along with probabilities
    preds = model.predict(img)
    pred = np.argmax(preds, axis=1)
    labels = dict((v,k) for k,v in class_indices.items())
    predictions = [labels[k] for k in pred][0]
    prob = np.max(preds, axis=1)[0]
    return {'Label': predictions, 'Probability': prob}


predict_single('https://xcdn.next.co.uk/common/Items/Default/Default/Publications/G85/shotview-315x472/8164/M05-222s.jpg')

In [ ]:
json.load(open('./models/class_indices.json'))